# Python's Class Development Toolkit
https://www.youtube.com/watch?v=HTLu2DFOdTg


# Summary
1. Inherit from `object`
1. __Instance variables__ for information unique to an instance
1. __Class variables__ for data shared among all instances
1. __Regular methods__ need `self` to operate on instance data
1. __Class methods__ implement alternative constructors. They need `cls` so they can create subclass instances as well.
1. __Static methods__ attach functions to classes. They don't need either `self` or `cls. Static methods improve discoverability and require context to be specified.
1. `__var` for class local reference. 
1. A __property__ lets getter and setter methods be invoked automatically by attribute access. This allows Python classes to freely expose their instance variables.
1. The `__slots__` variable implements the Flyweight Design Patter by suppressing instance dictionaries.

## Circle Company
- Agile Methodology
    - core idea: iterate and adapt quickly
    - out sith waterfall: design code test ship
    - Let little bits of design, coding and testing; inform later bits of design, docing and testing

- Lean Startup Methodology
    - Lean startup == Agile applied to business

### docstring: Company name and elevator pitch
* (module level) __docstring__

```Python
''' Circuitous, LLC - 
    An advanced Circle Analytics Company
'''
```

### New Style class: Circle class
* Start with __documentation__ 
    - Can generate PDF and review by business
* (class level) __docstring__
* New style classes
* Initialize instance variables
    - Init isn't a constructor. It's job is to initialize the instance variables.
    - `self` is your instance. It has already been made by the time it gets called.
    - Anything not unique to an instance does NOT go to instance variable
* Regular method
    - Regular methods have `self` as first argument
* Modules for code reuse:
    - `return 3.14 * self.radius ** 2.0` -> `return math.pi * self.radius ** 2.0`
* Class variables for shared data
    - shared by all the instances

In [3]:
import math

class Circle(object):
    'An advanced circle analytic toolkit'

    version = '0.1' # class variable

    def __init__(self, radius):
        self.radius = radius # instance variable

    def area(self):
        'Perform quadrature on a shape of uniform radius'
        return math.pi * self.radius ** 2.0

### Minimum viable product: Ship it!

```Python
# Tutorial
print('circuituous version', Circle.version)
...
```

### First cusomer: Academia

In [30]:
from random import random, seed

seed(123457) # Reproducable
print(f'Using Circutuous\N{trade mark sign} version {Circle.version}')
n = 10
circles = [Circle(random()) for i in range(n)]

avg = sum([c.area() for c in circles]) / n
print(f'The average area of {n} random circles is {avg:0.1f}')


Using Circutuous™ version 0.1
The average area of 10 random circles is 0.9


### Next Customer wants a perimeter method
- Second customer: Rubber sheet company

- how do we feel about exposing the radius attribute?
    - i.e. radius isn't constant after creating the instance
    - `getter` and `setter`
    - Yes, expose attributes in Python

In [44]:
import math

class Circle(object):
    'An advanced circle analytic toolkit'

    version = '0.2' # class variable

    def __init__(self, radius):
        self.radius = radius # instance variable

    def area(self):
        'Perform quadrature on a shape of uniform radius'
        return math.pi * self.radius ** 2.0
    
    def perimeter(self):
        return 2.0 * math.pi * self.radius

cuts = [0.1, 0.7, 0.8]
circles = [Circle(r) for r in cuts]
for c in circles:
    print(f'A circlet with a radius of {c.radius} has a perimeter of{c.perimeter()}')
    print(f'and a cold area of {c.area()}')
    c.radius *= 1.1
    print(f'and a warm area of {c.area()}\n')

A circlet with a radius of 0.1 has a perimeter of0.6283185307179586
and a cold area of 0.031415926535897934
and a warm area of 0.038013271108436504

A circlet with a radius of 0.7 has a perimeter of4.39822971502571
and a cold area of 1.5393804002589984
and a warm area of 1.8626502843133883

A circlet with a radius of 0.8 has a perimeter of5.026548245743669
and a cold area of 2.0106192982974678
and a warm area of 2.4328493509399363



### override: Third customer - National tire chain
* __extend__ method: parent method eventually get called
* __override__ method: parent method doesn't get called

In [55]:
class Tire(Circle):
    'Tires are circles with a corrected perimeter'
    
    def perimeter(self):
        'Circuference corrected for the rubber'
        return Circle.perimeter(self) * 1.25
    
t = Tire(22)
print(f'A tire of radius {t.radius} has an inner area of {t.area():.2f} '
      f'and an odometer corrected perimeter of {t.perimeter():.2f}\n')


A tire of radius 22 has an inner area of 1520.53 and an odometer corrected perimeter of 172.79



### classmethod: Next customer: National graphics company
Converting method
- The API is awkward. A converter function is always needed. Perhaps change the constructor signature? 
    ```Python
    bbd = 25.1
    c = Circle(bbd_to_radius(bbd))
    ...
    ```
- Constructor war: 
    - several people want to different signatures of constructor (e.g., most powerful customer)
    - Provide alternative constructors. Everyone should win.

    ```Python
    datatime(2019, 1, 20)
    datetime.fromtimestamp(1363493616)
    datetime.fromordinal(734000)
    datetime.now()

    dict.fromkeys(['k1', 'k2', 'k3'])
    ```
- __Class methods__ create alternative constructors
    - It should also work for subclasses. 
    - This code doesn't work:
    ```Python
    t = Tire.from_bbd(45)
    ```
- Subclasses?
    - Besure to use `cls` to support subclassing
    
    `return Circle(radius)` -> `return cls(radius)`


In [57]:
import math

class Circle(object):
    'An advanced circle analytic toolkit'

    version = '0.3' # class variable

    def __init__(self, radius):
        self.radius = radius # instance variable

    def area(self):
        'Perform quadrature on a shape of uniform radius'
        return math.pi * self.radius ** 2.0
    
    def perimeter(self):
        return 2.0 * math.pi * self.radius
    
    @classmethod            # alternative constructor
    def from_bbd(cls, bbd):
        'Construct a circle from a bounding box diagonal'
        radius = bbd / 2.0 / math.sqrt(2.0)
        # return Circle(radius)
        return cls(radius)


### staticmethod: New customer request: add a function
- Will this also work for the Sphere class and the Hyperbolic class? No / No
- Can people even find this code? 
    - No, it's not in the class
    - Indent
- No `self` used inside function body
- Purpose of `staticmethod`:
    * Attach functions to classes
    * Improve findability 
    * Ensure people using function in approproate context: `Circle.angle_to_grade(30)`

In [59]:
def angle_to_grade(angle):
    'Convert angle in degree to a percentage grade'
    return math.tan(math.radians(angle)) * 100.0

class Circle(object):
    'An advanced circle analytic toolkit'

    version = '0.4b' # class variable

    def __init__(self, radius):
        self.radius = radius # instance variable
        
    @staticmethod
    def angle_to_grade(angle):
        'Convert angle in degree to a percentage grade'
        return math.tan(math.radians(angle)) * 100.0


### Class local reference: Government request #1
Not allow to use radius in area calculation. Instead you must call `perimeter()` and back into the radius.

- Change in `p = self.perimeter()` doesn't work for children classes. 
- Subclass should be free to override any methods without breaking the code
- Solution 1: class local reference: keep a spare copy. `self._perimeter`
    - If someone overrides it, you still got the original
    - BUT Tire company also declares `_perimeter` 
    - Need to specify name: `_tireperimeter`, `_circleperimeter`
- Built in already: `self.__perimeter()`

__Class local reference__: making sure `self` is actually referring to _you_ instead of _you and your children_.
- Not about private
- All about freedom: make sure subclass is free to override any methods without breaking others.

In [68]:
class Circle(object):
    'An advanced circle analytic toolkit'

    version = '0.5b' # class variable

    def __init__(self, radius):
        self.radius = radius # instance variable

    def area(self):
        'Perform quadrature on a shape of uniform radius'
        p = self.perimeter()
        r = p / math.pi / 2.0
        return math.pi * r ** 2.0
    
    def perimeter(self):
        return 2.0 * math.pi * self.radius
    
    
# Problem with the tire company: perimeter scales 


    def area(self):
        'Perform quadrature on a shape of uniform radius'
        p = self._perimeter()
        r = p / math.pi / 2.0
        return math.pi * r ** 2.0
    _perimeter = perimeter
    

class Circle(object):
    'An advanced circle analytic toolkit'

    version = '0.5' # class variable

    def __init__(self, radius):
        self.radius = radius # instance variable

    def area(self):
        'Perform quadrature on a shape of uniform radius'
        p = self.__perimeter()
        r = p / math.pi / 2.0
        return math.pi * r ** 2.0
    
    def perimeter(self):
        return 2.0 * math.pi * self.radius
    
    __perimeter = perimeter # Auto renamed to _Circle__perimeter

print(Circle.__dict__)

{'__module__': '__main__', '__doc__': 'An advanced circle analytic toolkit', 'version': '0.5', '__init__': <function Circle.__init__ at 0x7fbc445dd840>, 'area': <function Circle.area at 0x7fbc445ddf28>, 'perimeter': <function Circle.perimeter at 0x7fbc443d9378>, '_Circle__perimeter': <function Circle.perimeter at 0x7fbc443d9378>, '__dict__': <attribute '__dict__' of 'Circle' objects>, '__weakref__': <attribute '__weakref__' of 'Circle' objects>}


### property: Government request #2
    - not allowed to store the _radius_
    - must store the _diameter_ instead

- `property`: Convert attribute access to method access

In [67]:
class Circle(object):
    'An advanced circle analytic toolkit'

    version = '0.6' # class variable

    def __init__(self, radius):
        self.radius = radius # instance variable
    
    @property               # convert dotted access to method calls
    def radius(self):
        'Radius of a circle'
        return self.diameter / 2.0
    
    @radius.setter
    def radius(self, radius):
        self.diameter = radius * 2.0

### slots: User request: Many circles
    - n = 10000000
    - major memory problem, each instance is over 300 bytes each (dict)
- Flyweight design pattern: `__slots__`
    - just allocate one pointer for the _diameter_ and nothing else.
    - lost `__dict__`
        - Lost ability to inspect `__dict__`
        - Can't add additional attributes
    - Save optimization  for the last
- If __subclassing__, `__slots__` does NOT inherent

In [1]:
class Circle(object):
    'An advanced circle analytic toolkit'

    __slots__ = ['diameter']
    version = '0.7' # class variable

    def __init__(self, radius):
        self.radius = radius # instance variable
    
    @property               # convert dotted access to method calls
    def radius(self):
        'Radius of a circle'
        return self.diameter / 2.0
    
    @radius.setter
    def radius(self, radius):
        self.diameter = radius * 2.0
        
    def area(self):
        'Perform quadrature on a shape of uniform radius'
        p = self.__perimeter()
        r = p / math.pi / 2.0
        return math.pi * r ** 2.0
    
    def perimeter(self):
        return 2.0 * math.pi * self.radius
    
    __perimeter = perimeter # Auto renamed to _Circle__perimeter
print(Circle.__dict__)

{'__module__': '__main__', '__doc__': 'An advanced circle analytic toolkit', '__slots__': ['diameter'], 'version': '0.7', '__init__': <function Circle.__init__ at 0x7f8ad00d0d08>, 'radius': <property object at 0x7f8ad014fc78>, 'area': <function Circle.area at 0x7f8ad00d0840>, 'perimeter': <function Circle.perimeter at 0x7f8ad00fc048>, '_Circle__perimeter': <function Circle.perimeter at 0x7f8ad00fc048>, 'diameter': <member 'diameter' of 'Circle' objects>}


In [2]:
from random import random, seed

n = 10000
seed(123457) # Reproducable
print(f'Using Circutuous\N{trade mark sign} version {Circle.version}')

circles = [Circle(random()) for i in range(n)]

avg = sum([c.area() for c in circles]) / n
print(f'The average area of {n} random circles is {avg:0.1f}')

Using Circutuous™ version 0.7


NameError: name 'math' is not defined

- class is a namespace; defination runs as if it runs in module and the module dictionary becomes the class dictionary
- YAGNI - You Aint Gonna Need It
- When finished one class, polish and think about the subclasses come after this

# Subclassing
https://www.youtube.com/watch?v=miGolgp9xq
The Art of Subclassing

In [3]:
# Simple example of subclassing

class Animal(object):
    'Generic animal class'
    
    def __init__(self, name):
        self.name = name
        
    def walk(self):
        print(f'{self.name} is Walking')
        
class Dog(Animal):
    'Man\'s best friend'
    
    def bark(self):
        print('Woof')

## Terminology

- Adding
    - adding new method, new capability
    - Dog adds the `bark()` method to Animal which didn't have such method.
- Overriding
    - replacing existing method in subclass
    - Snake repaces the `walk()` method of Animal with a `walk()` method that knows how to slither.
- Extending
    - inheritave capability AND adding new behaviour
    - Cat modifies the `walk()` method to add tail swishing behavior while delegating the work of actual walking to its parent.

## Pattern for Subclassing

###  Frameworks

- The parent class supplies all of the "controller" functionality and makes calls to pre-named stub methods
- the subclass overrides stub methods of interest
    - to a more concreate / specific use
- For example, SimpleHTTPServer runs an event loop and dispatches HTTP requests to stub methods like `do_HEAD()` and `do_GET()`
    - no actual implementation in parent class
- Someone writing an HTTP server would use a subclass to supply the desired actions in the event of a `GET` or `HEAD` request
- Very static

### Dynamic dispatch to subclass methods

- Subclasser has greater control
- The parent class uses getattr() to dispatch to new functionality
- The child class implements appropriately named methods
- Example from cmd.py:

```Python
def onecmd(self, cmd, arg):
    try:
        func = getattr(self, 'do_' + cmd)
    except AttributeError:
        return self.default(cmd)
    return func(arg)
```
- How it might be used in a subclass:
    - user specify 'pendown' and code call `do_pendown` for you.
    
```Python
def do_pendown(self):
    self.canvas.setpen(1)
```    

## Call Patterns for subclassing


In [10]:
import math

class Circle:
    def __init__(slef, radius):
        self.radius = radius
    def area(self):
        return self.radius ** 2.0 * math.pi
    def __repr__(self):
        return f'{self.__class__.__name__} has area {self.area()}'
    
class Donut(Circle):
    def __init__(self, outer, inner):
        Circle.__init__(self, outer)
        self.inner = inner
    def area(self):
        outer, inner = self.radius, self.inner
        return Circle(outer).area() - Circle(inner).area()

- subclass calls parent's `__init__` (vs. super())
    - extending `__init__`
- Replace `area` with a different 
    - override `area`
- subclass doesn't have `__repr__`, going up looking for `__repr__`
    - Circle's `__repr__` is running on Donut
    - Didn't hardware class name, using `__class__.__name__`
    - `self.area()` - Circle or Donut? 
        - Donut -> `__repr__` in Circle -> `self.area()` in Donut
    - __Everytime__ you refer to 'self', going to bottom and going up again.
        
    - similarly, calling `super()` might end up with someone unrelated
        - go up from children
        - call other's inheritance tree

## About Subclasses
### An object or class? 
- One common definition of an object:

    An entity that encapsulates data together with functions (methods) for manipulating that data


- Operationally in Python, we implement that with dictionaries
    - __Instance dictionaries__ hold state and point to their classes
    - __Class dictionaries__ hold the functions (methods)
    
```
 InstDict1  +----------+
                       |
                       v
 InstDict2  +-----> ClassDict
                       ^
                       |
 InstDict3 +-----------+

```
### Subclass? 

- A subclass is just a class that __delegates work__ to another class
- A subclass nad its parent are just two different dictionaries that contain functions
- A subclass points to its parent
- The pointer means " _I delegate work to this class_ "
    - not "subclass specialize..."
    - parent is the subcontractor

```
 InstDict1  +----------+
                       |
                       v
 InstDict2  +-----> SubClassDict +----> ParClassDict
                       ^
                       |
 InstDict3 +-----------+
```

- In other words, subclassing can be viewed as a technique for _code re-use_
- It is the __subclass that is in charge__
- The subclass decides what work gets delegated

### Operational view vs the conceptual view

__Operational__ view of subclassing:
- Classes are dictionaries of functions
- Subclasses point to other dictionaries to reuse their code
    - subclass decides what work it contracts to
- Subclasses are in complete control of what happens

__Conceptual__ views of subclass:
- Parent classes define an interface
- Subclasses can extend that interface
- Parents are in charge
- Subclasses are just specializations of the parent
    - Dog is an instance of Animal that knows how to bark
    - Counter is an instance of dict that has a default of zero
    - A named tuple is an instance of tuple that also has attribute access
 

## Liskov Substitution Principle
If S is a subtype of T, then objects of type T may be replaced with objects of the S

```Python
def name_pet(animal):
    print(f'The pet\'s name is {animal.name}')
    
name_pet(Animal('Polly'))
name_pet(Dog('Fido'))
```

- It's all about _polymorphism_ and substitutability_ so that the subclass can be used in client code without changing the client code.

### Liskov Violations
- Any part of the API that isn't fully substitutable
- This is common and normal
- In particular, useful subclasses commonly have _different constructor signatures_
- For example, the array API is very similar to the list API but the constructor is different => Not substitutable
```Python
    s = list(some_iter)
    s = array('c', some_iter)
```
- Minimize impact
            
### Goal: isolate or minimize the impact
- MutableSet instances support `union()`, `intersection()`, `difference()`
- they need to be able to create new instances of MutableSets
- but the signature of the constructor is unknown
- so we factor out calls to the constructor in `_from_iterable()`
    - as an adapter
    - Call this instead of default constructor
    
```Python
class TypeSet(MutableSet):
    def __init__(self, thetype, iterable):
        self.thetype = thetype
        ...
    
        def _from_iterable(self, iterable):
            'adapter for the constructor'
            it = iter(iterable)
            first = next(it)
            return TypeSet(type(first), iterable)
```
- get union, intersect and differ for free by overriding `_from_iterable`

### The Circle / Ellipse Problem
- no right / wrong answer
- subclasses reuse code in parent
- i.e., parent should have the __most reusable code__


- `exception` problem: 
    - exception like runtimeerror, valueerror, typeerror
    - solution: multiple inheritance

## Open-Closed Principle
- "Software entities (classes, modules, functions, etc.) should be open for extension, but closed for modification"
- many different interpretations
    - (Java) use ABC to create fixed interfaces with multiple implementations
    - objects have internal invariants and that subclasses shouldn't be able to break those invariants
    - i.e., the classes capabilities can be extended but the underlying class sohuldn't get broken

In [12]:
class CaseInsensitiveDict(dict):
    def __setitem__(self, key, value):  # override setitem
        key = key.lower()
        dict.__setitem__(self, key, value)
    def __getitem__(self, key):         # override getitem
        key = key.lower()
        return dict.__getitem__(self, key)
# doesn't work
# Overriding getitem isn't sufficient overriding `get`


- dict has internal invariants
- need to override every methods

### OCP in Python with name mangling
A method called `__update` in a class called MyDict transforms the name into `_MyDict__update`

Class local reference

This makes the method invisible to subclasses. Use this to create _protected internal callse_ in addition to overridable public methods.

```Python
class MyDict:
    def __init__(self, iterable):
        self.items_list = []
        self.__update(iterable)
    def update(self, iterable):
        for item in iterable:
            self.items_list.append(item)
    __update = update
```